<a href="https://colab.research.google.com/github/loshantj/Brain_Tumor_Detection/blob/main/Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!nvidia-smi

Tue Aug 22 20:19:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Lambda, Flatten,Dense,GlobalAveragePooling2D

import numpy as np

In [3]:
IMAGE_SIZE=[224,224]

train_path="/content/drive/MyDrive/Datathon-Dataset/train"
test_path="/content/drive/MyDrive/Datathon-Dataset/test"

In [5]:
inception= InceptionV3(input_shape=IMAGE_SIZE+[3],weights="imagenet",include_top=False)

In [6]:
for layer in inception.layers:
    layer.trainnable=False

In [7]:
from glob import glob

In [8]:
folders=glob("/content/drive/MyDrive/Datathon-Dataset/train/**")

In [9]:
folders


['/content/drive/MyDrive/Datathon-Dataset/train/no_tumor',
 '/content/drive/MyDrive/Datathon-Dataset/train/category3_tumor',
 '/content/drive/MyDrive/Datathon-Dataset/train/category1_tumor',
 '/content/drive/MyDrive/Datathon-Dataset/train/category2_tumor']

In [10]:
x=inception.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)

In [11]:
prediction=Dense(len(folders),activation='softmax')(x)

In [12]:
model=Model(inputs=inception.input, outputs=prediction)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                             

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [16]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)



In [17]:
test_set=test_datagen.flow_from_directory("/content/drive/MyDrive/Datathon-Dataset/test/",
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

Found 80 images belonging to 4 classes.


In [19]:
training_set=train_datagen.flow_from_directory("/content/drive/MyDrive/Datathon-Dataset/train/",
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

Found 320 images belonging to 4 classes.


In [20]:
my_model=model.fit(
    training_set,
    validation_data=test_set,
    epochs=20,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/20
10/10 [==============================] - 139s 8s/step - loss: 1.7429 - accuracy: 0.5281 - val_loss: 8.7510 - val_accuracy: 0.2500
Epoch 2/20
10/10 [==============================] - 6s 604ms/step - loss: 0.6577 - accuracy: 0.7906 - val_loss: 16.3492 - val_accuracy: 0.2500
Epoch 3/20
10/10 [==============================] - 6s 564ms/step - loss: 0.3685 - accuracy: 0.8750 - val_loss: 5.9496 - val_accuracy: 0.2500
Epoch 4/20
10/10 [==============================] - 7s 662ms/step - loss: 0.2389 - accuracy: 0.9281 - val_loss: 4.1267 - val_accuracy: 0.1875
Epoch 5/20
10/10 [==============================] - 6s 539ms/step - loss: 0.2532 - accuracy: 0.9250 - val_loss: 4.9583 - val_accuracy: 0.2500
Epoch 6/20
10/10 [==============================] - 7s 735ms/step - loss: 0.2984 - accuracy: 0.9156 - val_loss: 2.8939 - val_accuracy: 0.2250
Epoch 7/20
10/10 [==============================] - 6s 551ms/step - loss: 0.1810 - accuracy: 0.9438 - val_loss: 2.5911 - val_accuracy: 0.1500
Epoch 

In [22]:
my_model

In [25]:
model.save("inceptionv3_model.h5")


In [26]:
from tensorflow.keras.models import load_model

loaded_model = load_model("inceptionv3_model.h5")


In [46]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
from PIL import Image
import numpy as np

def preprocess_and_resize_image(uploaded_image_path, target_size):
    # Open and resize the uploaded image
    img = Image.open(uploaded_image_path)
    img = img.resize(target_size)

    # Convert image to array and preprocess
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)

    return img_array


In [48]:
def predict_tumor_category(image_array, model):
    predictions = model.predict(image_array)
    categories = ["category1_tumor", "category2_tumor", "category3_tumor", "no_tumor"]
    predicted_category = categories[np.argmax(predictions)]
    return predicted_category


In [72]:
from google.colab import files

# Allow the user to upload an image
uploaded = files.upload()

# Get the first uploaded file (assuming there's only one)
uploaded_image_path = next(iter(uploaded))

# Set the target size for resizing
target_size = (224, 224)

# Preprocess and resize the uploaded image
image_array = preprocess_and_resize_image(uploaded_image_path, target_size)

# Make predictions on the resized image
predicted_category = predict_tumor_category(image_array, loaded_model)
print("Predicted Tumor Category:", predicted_category)


Saving pt_img (891).jpg to pt_img (891) (3).jpg
1/1 [==============================] - 0s 27ms/step
Predicted Tumor Category: category3_tumor
